# Algorytm Arnoldiego

$AQ_m = Q_m H_m + $ reszta, gdzie $Q_m = [q_1, ..., q_m]$ i $H_m$ Hessenberga

$q_1 := \frac{x_0}{||x_0||_2}$

$h_{1,1} := q_1^T A q_1$

$s := Aq_1 - h_{1,1} q1$

$h_{2,1} := ||s||_2$

$q_2 := s/h_{2,1}$

for $j = 2:m$ begin

$h_{j,j} := q_j^T A q_j$

$s := Aq_j - \sum_{i=1}^{j} h_{ij}q_i$

$h_{j+1,j} := ||s||_2$

$q_{j+1} := s / h_{j+1,j}$

end



# Algorytm Lanczosa

$AQ_m = Q_m H_m + $ reszta, gdzie $Q_m = [q_1, ..., q_m]$ i $T_m = H_m$ trójdiagonalna: $\alpha$ na diagonali, $\beta$ poza

$q_1 := x_0/||x_0||_2$

$\beta_0 := 0$

for $j = 1:m$ begin

$w := A q_j$

$\alpha_j := q_j^T w$

$s := w - (\alpha_j q_j + \beta_{j-1} q_{j-1})$

$\beta_j := ||s||_2$

$q_{j+1} := s / \beta_j$

end


In [620]:
import numpy as np

In [621]:
def arnoldi(A):
    n = A.shape[0]
    Q = np.zeros((n, n))
    H = np.zeros((n, n))
    Q[:, 0] = np.random.rand(n)
    Q[:, 0] = Q[:, 0] / np.linalg.norm(Q[:, 0])
    for j in range(n):
        v = A @ Q[:, j]
        for i in range(j+1):
            H[i, j] = Q[:, i].T @ v
            v = v - H[i, j] * Q[:, i]
        if j < n-1:
            H[j+1, j] = np.linalg.norm(v)
            Q[:, j+1] = v / H[j+1, j]
    return Q, H

In [622]:
def lanczos(A, m):
    # można jeszcze dołożyć reortogonalizację
    n = A.shape[0]
    Q = np.zeros((n, m))
    Q[:, 0] = np.random.rand(n)
    Q[:, 0] /= np.linalg.norm(Q[:, 0])
    alpha = np.zeros(m)
    beta = np.zeros(m - 1)
    for j in range(m):
        w = A @ Q[:, j]
        alpha[j] = Q[:, j].T @ w
        s = w - alpha[j] * Q[:, j] - (np.zeros(m) if j == 0 else beta[j - 1] * Q[:, j - 1])
        if j < m - 1:
            beta[j] = np.linalg.norm(s)
            Q[:, j + 1] = s / beta[j]
    T = np.diag(alpha) + np.diag(beta, 1) + np.diag(beta, -1)
    return Q, T


In [623]:
N = 15
A = np.random.rand(N, N)
# d = np.random.rand(N)
# A = np.diag(d)
A = A + A.T

In [624]:
m = N
Q, T = lanczos(A, m)
np.linalg.norm(A @ Q - Q @ T)

6.788774398976261

In [625]:
Q2, H = arnoldi(A)
np.linalg.norm(A @ Q2 - Q2 @ H)

3.010123896634058e-15

In [626]:
np.linalg.norm(Q.T @ Q - np.identity(m))

1.15299710223616

In [627]:
sorted(np.linalg.eig(T)[0])

[-2.76840475967154,
 -2.075270328815419,
 -1.8945722409692638,
 -0.9697193864616862,
 -0.5610826345877413,
 -0.022299505740645032,
 0.4863022788678264,
 0.688487005922288,
 0.970040049320727,
 1.2592851867810833,
 1.6927308982639746,
 2.112785841439869,
 2.5119620223618284,
 10.193833702685543,
 14.974821722513]

In [628]:
sorted(np.linalg.eig(H)[0])

[-2.7684047596716805,
 -2.0752703297464463,
 -1.89457224398687,
 -0.9697196085398518,
 -0.5610843464866492,
 -0.13305256278584116,
 -0.011263787731072037,
 0.5614770735417186,
 0.9652112826783618,
 1.258459873509545,
 1.416421575124967,
 1.692737145046271,
 2.1128466723743577,
 2.5119648211262877,
 14.974821722512988]

In [629]:
sorted(np.linalg.eig(A)[0])

[-2.7684047596716708,
 -2.0752703297464477,
 -1.8945722439868669,
 -0.9697196085398531,
 -0.5610843464866508,
 -0.13305256278584074,
 -0.01126378773107197,
 0.5614770735417169,
 0.9652112826783604,
 1.2584598735095445,
 1.416421575124967,
 1.6927371450462678,
 2.112846672374364,
 2.5119648211262886,
 14.974821722513003]